# Parameters significance $\rightarrow$ T-values
Este notebook ha sido adaptado para su uso en **Google Colab**.

🔁 Si necesitas subir archivos, usa `from google.colab import files`

📈 Visualiza gráficos directamente con `matplotlib.pyplot`.

✅ Asegúrate de ejecutar cada celda en orden para reproducir los resultados correctamente.

In [1]:
🛠️ Instalar todos los paquetes necesarios
!pip install -q numpy pandas matplotlib scipy sympy tabulate
from google.colab import drive
drive.mount('/content/drive')

SyntaxError: invalid character '🛠' (U+1F6E0) (170188540.py, line 1)

## Libraries

In [1]:
import sympy as sp
from scipy.optimize import least_squares
from sympy.utilities.lambdify import lambdify
import numpy as np
from scipy.interpolate import interp1d
from scipy.integrate import solve_ivp
from joblib import Parallel, delayed
import warnings
from scipy.io import loadmat
import matplotlib.pyplot as plt
import pandas as pd
from tabulate import tabulate
from scipy.optimize import least_squares
plt.rcParams['font.family'] = 'Times New Roman'

## Base Case - No Fixed Parameters

In [ ]:
# --- Symbolic Definitions ---

# Time (t) and inputs (U_i)
t = sp.Symbol('t', real=True)
U1, U2 = sp.symbols('U1 U2', real=True)

# State variables
x = sp.symbols('x1 x2 x3 x4 x5', real=True)
statesSym = sp.Matrix(x)

# Free parameters
th = sp.symbols('th1 th2 th3 th4 th5 th6 th7 th8 th9 th10 th11 th12 th13 th14', real=True)
thetaSym = sp.Matrix(th)
th1, th2, th3, th4, th5, th6, th7, th8, th9, th10, th11, th12, th13, th14 = th

# Initial conditions
X0, N0, G0, F0, E0 = 0.2000, 0.2090, 115.55, 100.94, 0
x0 = [X0, N0, G0, F0, E0]
x0Sym = sp.Matrix(x0)
dx0dthSym = x0Sym.jacobian(thetaSym)

# Observed outputs
outputsSym = sp.Matrix([x[1], x[2], x[3]])
ny = outputsSym.shape[0]
dhdxSym = outputsSym.jacobian(statesSym)
dhdthSym = outputsSym.jacobian(thetaSym)

# --- ODE System ---
R = 8.314
x1, x2, x3, x4, x5 = statesSym
mu = th1 * sp.exp(59453 * (U1 - 300) / (300 * R * U1)) * (x2 / (x2 + th4 * sp.exp(46055 * (U1 - 293.15) / (293.15 * R * U1))))
death = U2 * th9 * sp.exp(0.0415 * x5 + (130000 * (U1 - 305.65)) / (305.65 * R * U1))

Xdot = sp.Matrix([
    mu * x1 - death * x1,
    -mu * (x1 / th10),
    -((mu / th11) + (th2 * sp.exp(11000 * (U1 - 296.15) / (296.15 * R * U1)) * (x3 / (x3 + th5 * sp.exp(46055 * (U1 - 293.15) / (293.15 * R * U1)))) * (th8 * sp.exp(46055 * (U1 - 293.15) / (293.15 * R * U1)) / (x5 + th8 * sp.exp(46055 * (U1 - 293.15) / (293.15 * R * U1)))) / th13) + 0.01 * sp.exp(37681 * (U1 - 293.3) / (293.3 * R * U1)) * (x3 / (x3 + x4))) * x1,
    -((mu / th12) + (th3 * sp.exp(11000 * (U1 - 296.15) / (296.15 * R * U1)) * (x4 / (x4 + th6 * sp.exp(46055 * (U1 - 293.15) / (293.15 * R * U1)))) * (th7 * sp.exp(46055 * (U1 - 293.15) / (293.15 * R * U1)) / (x3 + th8 * sp.exp(46055 * (U1 - 293.15) / (293.15 * R * U1)))) * (th8 * sp.exp(46055 * (U1 - 293.15) / (293.15 * R * U1)) / (x5 + th8 * sp.exp(46055 * (U1 - 293.15) / (293.15 * R * U1)))) / th14) + 0.01 * sp.exp(37681 * (U1 - 293.3) / (293.3 * R * U1)) * (x4 / (x3 + x4))) * x1,
    (th2 * sp.exp(11000 * (U1 - 296.15) / (296.15 * R * U1)) * (x3 / (x3 + th5 * sp.exp(46055 * (U1 - 293.15) / (293.15 * R * U1)))) * (th8 * sp.exp(46055 * (U1 - 293.15) / (293.15 * R * U1)) / (x5 + th8 * sp.exp(46055 * (U1 - 293.15) / (293.15 * R * U1)))) + th3 * sp.exp(11000 * (U1 - 296.15) / (296.15 * R * U1)) * (x4 / (x4 + th6 * sp.exp(46055 * (U1 - 293.15) / (293.15 * R * U1)))) * (th7 * sp.exp(46055 * (U1 - 293.15) / (293.15 * R * U1)) / (x3 + th8 * sp.exp(46055 * (U1 - 293.15) / (293.15 * R * U1)))) * (th8 * sp.exp(46055 * (U1 - 293.15) / (293.15 * R * U1)) / (x5 + th8 * sp.exp(46055 * (U1 - 293.15) / (293.15 * R * U1))))) * x1
])

# --- Jacobians ---
dfdx = Xdot.jacobian(statesSym)
dfdth = Xdot.jacobian(thetaSym)
dx0dth = sp.Matrix(x0).jacobian(thetaSym)

# --- Lambdify ---
x0_func = lambdify(thetaSym, x0Sym, modules='numpy')
dx0_func = lambdify((statesSym, thetaSym), dx0dthSym, modules='numpy')
h_func = lambdify((t, *statesSym, *thetaSym), outputsSym, modules='numpy')
dhdx_func = lambdify((t, *statesSym, *thetaSym), dhdxSym, modules='numpy')
dhdth_func = lambdify((t, *statesSym, *thetaSym), dhdthSym, modules='numpy')

model_funcs = {
    "f": lambdify((t, *statesSym, U1, U2, *thetaSym), Xdot, modules='numpy'),
    "dfdx": lambdify((t, *statesSym, U1, U2, *thetaSym), dfdx, modules='numpy'),
    "dfdth": lambdify((t, *statesSym, U1, U2, *thetaSym), dfdth, modules='numpy'),
}


# --- Load Data ---
data = loadmat(r"/content/drive/MyDrive/ColabNotebooks/Identifiability_Python/Operational_Data.mat")
Time_Temp_Pairs = data['Time_Temp_Pairs']
TU = Time_Temp_Pairs.copy()
TU[:, 1] += 273.15


### Functions 

In [3]:
# --- Meta System ---
def meta(t, x, TU, theta, model_funcs, dim):
    """
    Evaluates the extended ODEs system with parameter sensitivity.
    Inputs:
    -----------
    t : float
        Current integration time.
    x : ndarray
        Extended vector [x; dx/dtheta].
    TU : ndarray or None
        Operational conditions matrix. TU[:,0]=time, TU[:,1]=temperature.
    theta : ndarray
        Nominal vector values of the parameters.
    model_funcs : dict
        Dictionary with the symbolic functions:
        {"f": f_func, "dfdx": dfdx_func, "dfdth": dfdth_func}
    dim : tuple
        System dimensions: (n_states, n_parameters).

    Outputs:
    --------
    dxdt : ndarray
        Extended system derivative.
    """
    nx, nth = dim
    X = x[:nx]
    dxdth = x[nx:].reshape((nx, nth))

    if TU is not None:
        U1 = float(interp1d(TU[:, 0], TU[:, 1], kind='nearest', fill_value="extrapolate")(t))
        Td = -0.0001 * X[4]**3 + 0.0049 * X[4]**2 - 0.1279 * X[4] + 315.89
        U2 = 1.0 if U1 >= Td else 0.0
    else:
        U1 = 0.0
        U2 = 0.0

    f_func     = model_funcs["f"]
    dfdx_func  = model_funcs["dfdx"]
    dfdth_func = model_funcs["dfdth"]

    xdot       = np.array(f_func(t, *X, U1, U2, *theta)).flatten()
    dfdx       = np.array(dfdx_func(t, *X, U1, U2, *theta)).reshape((nx, nx))
    dfdth      = np.array(dfdth_func(t, *X, U1, U2, *theta)).reshape((nx, nth))

    dxdth_dot  = dfdx @ dxdth + dfdth

    return np.concatenate([xdot, dxdth_dot.flatten()])


# --- Auxiliary Function: T-value Analysis ---
def analizar_t_values_multiples(residuals_func, p0, bounds, texp, ydata, fixed_idx=None, n_iter=10):
    from scipy.optimize import least_squares
    n_params = len(p0)
    t_values_all = []

    for i in range(n_iter):
        p_init = np.random.uniform(bounds[0], bounds[1])
        if fixed_idx is not None:
            p_init[fixed_idx] = p0[fixed_idx]

        result = least_squares(
            residuals_func,
            p_init,
            args=(texp, ydata),
            bounds=bounds,
            jac='2-point'
        )

        params_hat = result.x
        J = result.jac
        resid = result.fun

        sigma2 = np.sum(resid**2) / (len(ydata) - np.linalg.matrix_rank(J))
        try:
            Cov_theta = sigma2 * np.linalg.pinv(J.T @ J)
            std_theta = np.sqrt(np.diag(Cov_theta))
            std_theta[std_theta == 0] = np.nan
            t_values = params_hat / std_theta
            t_values_all.append(t_values)
        except np.linalg.LinAlgError:
            print(f"[{i+1}] Singular matrix, skipping this iteration.")
            continue

    t_values_all = np.array(t_values_all)
    t_avg = np.nanmean(t_values_all, axis=0)
    t_std = np.nanstd(t_values_all, axis=0)

    return t_avg, t_std, t_values_all

In [4]:
# --- Residuals Function with Corrections ---
def residuals(theta_est, *args):
    theta_full = theta_nominal_completo.copy()
    theta_full[:len(theta_est)] = theta_est  # Insert estimated values

    try:
        x0 = np.array(x0_func(*theta_full)).flatten()
        dx0 = np.array(dx0_func(x0.reshape(-1, 1), theta_full)).reshape(x0.shape[0], -1)
        x_init = np.concatenate([x0, dx0.flatten()])

        sol = solve_ivp(lambda t, x: meta(t, x, TU, theta_full, model_funcs, (nx, len(theta_full))),
                        (0, Tf), x_init, t_eval=t_eval,
                        rtol=1e-2, atol=1e-4, method='BDF')

        if not sol.success:
            return np.ones(Nt * ny) * 1e6

        Xsol = sol.y[:nx, :].T
        y_model = np.zeros((Nt, ny))
        for i in range(Nt):
            inputs = list(Xsol[i]) + list(theta_full)
            y_i = np.array(h_func(t_eval[i], *inputs)).flatten()
            y_model[i, :] = y_i

        return (y_model - y_exp_interp).flatten()

    except Exception as e:
        print(f"Error: {e}")
        return np.ones(Nt * ny) * 1e6


# --- t-value Calculation Function ---
def t_values(residuals_func, theta_0, bounds, texp, ydata):
    resultado = least_squares(
        residuals_func,
        theta_0,
        args=(texp, ydata),
        bounds=bounds,
        jac='2-point',
        method='trf'  # Cambiado para mayor robustez con límites
    )

    theta_hat = resultado.x
    J = resultado.jac
    res = resultado.fun

    dof = len(res) - np.linalg.matrix_rank(J)
    sigma2 = np.sum(res**2) / dof if dof > 0 else 1.0

    try:
        Cov_theta = sigma2 * np.linalg.pinv(J.T @ J)
        std_theta = np.sqrt(np.diag(Cov_theta))
    except np.linalg.LinAlgError:
        Cov_theta = np.full((len(theta_hat), len(theta_hat)), np.nan)
        std_theta = np.full(len(theta_hat), np.nan)

    with np.errstate(divide='ignore', invalid='ignore'):
        t_values = theta_hat / std_theta
        t_values[np.isnan(t_values)] = 0

    return theta_hat, t_values, std_theta, Cov_theta


def repeated_t_values(
    residuals_func, theta_0, bounds, texp, ydata,
    n_reps=10, verbose=False
):
    """
    Ejecuta múltiples ajustes para calcular t-values y estadísticas.
    También retorna cuántas veces cada parámetro fue significativo (|t| > 2).
    """
    t_matrix = []

    for i in range(n_reps):
        theta_init = np.random.uniform(bounds[0], bounds[1])
        if verbose:
            print(f"[{i+1}] Ejecutando ajuste...")

        try:
            _, t_vals, _, _ = t_values(
                residuals_func=residuals_func,
                theta_0=theta_init,
                bounds=bounds,
                texp=texp,
                ydata=ydata
            )
            t_matrix.append(t_vals)
        except Exception as e:
            if verbose:
                print(f"Error en repetición {i+1}: {e}")
            continue

    t_matrix = np.array(t_matrix)
    t_avg = np.nanmean(t_matrix, axis=0)
    t_std = np.nanstd(t_matrix, axis=0)
    t_var = np.nanvar(t_matrix, axis=0)
    t_sig_count = np.sum(np.abs(t_matrix) > 2, axis=0)  # <- Nuevo: conteo significancia

    return t_avg, t_std, t_var, t_matrix, t_sig_count


In [5]:
# --- Paralelized Function for Multiple Combinations ---
def _simular_t_values(fijados, i, theta_0, bounds, texp, ydata, param_names, residuals_func, n_reps):
    try:
        fijados_idx = [param_names.index(p) for p in fijados]
    except ValueError as e:
        print(f"Parámetro no encontrado: {e}")
        return None

    libres_idx = [i for i in range(len(theta_0)) if i not in fijados_idx]
    libres_names = [param_names[i] for i in libres_idx]

    t_matrix = []

    for _ in range(n_reps):
        theta_init = np.random.uniform(bounds[0], bounds[1])
        theta_init[fijados_idx] = theta_0[fijados_idx]

        try:
            _, t_vals, _, _ = t_values(
                residuals_func=residuals_func,
                theta_0=theta_init,
                bounds=bounds,
                texp=texp,
                ydata=ydata
            )
            t_matrix.append(t_vals)
        except Exception as e:
            print(f"Error en simulación con {fijados}: {e}")
            continue

    t_matrix = np.array(t_matrix)
    t_mean = np.nanmean(t_matrix, axis=0)
    t_std = np.nanstd(t_matrix, axis=0)
    t_count = np.sum(np.abs(t_matrix) > 2, axis=0)

    return {
        "fijados": fijados,
        "libres": libres_names,
        "t_mean": t_mean,
        "t_std": t_std,
        "t_count": t_count,
        "n_reps": t_matrix.shape[0]
    }

# --- t-value Calculation Function with Multiple Combinations of Fixed Parameters ---
def calcular_t_values_multiple_combinaciones(
    residuals_func, theta_0, bounds, texp, ydata,
    combinaciones_fijadas, param_names,
    sim_per_combo, n_jobs
):
    tareas = [
        delayed(_simular_t_values)(
            fijados, i, theta_0, bounds, texp, ydata,
            param_names, residuals_func, sim_per_combo
        )
        for i, fijados in enumerate(combinaciones_fijadas)
    ]

    resultados = Parallel(n_jobs=n_jobs, verbose=10)(tareas)

    for res in resultados:
        if res is None:
            continue

        print("\n" + "=" * 80)
        print(f"🔒 Fijados: {', '.join(res['fijados']) if res['fijados'] else 'Ninguno'}")
        print(f"🔓 Libres: {', '.join(res['libres'])}")
        print(f"🔁 Iteraciones exitosas: {res['n_reps']} de {sim_per_combo}\n")

        tabla = []
        for pname, t_mean, t_std, t_cnt in zip(res["libres"], res["t_mean"], res["t_std"], res["t_count"]):
            tabla.append([
                pname,
                f"{t_mean:.3f}",
                f"{t_std:.3f}",
                f"{int(t_cnt)} / {res['n_reps']}",
                "✅" if abs(t_mean) > 2 else "—"
            ])

        print(tabulate(
            tabla,
            headers=["Parametro", "t promedio", "t std", "Significativo (#)", "Significativo (|t|>2)"],
            tablefmt="fancy_grid"
        ))


### t-values Calculation

In [ ]:
# --- Parameters for T-value Analysis ---
Tf = 161 # Total time for simulation in hours
Nt = 50 # Number of time points for evaluation
t_eval = np.linspace(0, Tf, Nt)

# --- Reading Experimental Data ---
archivo = r"/content/drive/MyDrive/ColabNotebooks/Identifiability_Python/Bioreactor2020-cineticafermentacion.xlsx"
sheet = "LAB-LO(02)"
df = pd.read_excel(archivo, sheet_name = sheet)
cols = ["YAN (mg/L)", "Glucosa (g/L)", "Fructosa (g/L)", "Time (min)"]
df = df.replace('-', np.nan)
df = df.dropna(subset=cols)
df[cols] = df[cols].astype(float)
df["Time (h)"] = df["Time (min)"] / 60.0
t_exp = df["Time (h)"].to_numpy()
exp_data = df.groupby("Time (h)")[["YAN (mg/L)", "Glucosa (g/L)", "Fructosa (g/L)"]].mean().reset_index()
y_exp_interp = np.zeros((len(t_eval), 3))

for j, col in enumerate(["YAN (mg/L)", "Glucosa (g/L)", "Fructosa (g/L)"]):
    if col == "YAN (mg/L)":
        y_exp_interp[:, j] = np.interp(t_eval, exp_data["Time (h)"]/1000, exp_data[col])
    else:
        y_exp_interp[:, j] = np.interp(t_eval, exp_data["Time (h)"], exp_data[col])


# --- Parameters (Nominal and Bounds) ---
theta_nominal_completo = np.array([
    0.18, 0.225, 0.225, 0.01, 7.5, 7.5,
    N0/4, 40, 0.00044, 19.69, 1.60, 1.60, 0.49, 0.49
])
thetaLow = 0.9 * theta_nominal_completo
thetaHigh = 1.1 * theta_nominal_completo

# nx must match the number of states
nx = 5

# Experimental data loading (make sure df is already defined)
t_exp = df["Time (h)"].to_numpy()
y_exp_interp = np.zeros((len(t_eval), 3))
t_data = df.groupby("Time (h)")[["YAN (mg/L)", "Glucosa (g/L)", "Fructosa (g/L)"]].mean().reset_index()

for j, col in enumerate(["YAN (mg/L)", "Glucosa (g/L)", "Fructosa (g/L)"]):
    y_exp_interp[:, j] = np.interp(t_eval, t_data["Time (h)"], t_data[col])

In [12]:
t_avg, t_std, t_var, t_vals_all, t_sig_count = repeated_t_values(
    residuals_func=residuals,
    theta_0=theta_nominal_completo,
    bounds=(thetaLow, thetaHigh),
    texp=t_exp,
    ydata=y_exp_interp.flatten(),
    n_reps=10,
    verbose=True
)

# Crear DataFrame con métricas
param_names = [f"θ{i+1}" for i in range(len(t_avg))]
significance = ["Yes" if abs(t) > 2 else "No" for t in t_avg]

df_stats = pd.DataFrame({
    "Parameter": param_names,
    "Mean t-value": t_avg,
    "Std. Dev.": t_std,
    "Variance": t_var,
    "Times Significant": t_sig_count,
    "Significant (|t| > 2)": significance
})

print(df_stats.round(3))
df_stats.to_excel("t_values_analysis.xlsx", sheet_name="T-Values", index=False)

[1] Ejecutando ajuste...


C:\Users\felip\AppData\Local\Temp\ipykernel_15404\591100218.py:52: RuntimeWarning: invalid value encountered in sqrt
  std_theta = np.sqrt(np.diag(Cov_theta))


[2] Ejecutando ajuste...
[3] Ejecutando ajuste...
[4] Ejecutando ajuste...
[5] Ejecutando ajuste...
[6] Ejecutando ajuste...
[7] Ejecutando ajuste...
[8] Ejecutando ajuste...
[9] Ejecutando ajuste...
[10] Ejecutando ajuste...
   Parameter  Mean t-value   Std. Dev.      Variance  Times Significant  \
0         θ1         1.932       5.445  2.965000e+01                  1   
1         θ2         1.088       3.173  1.006600e+01                  1   
2         θ3         0.229       0.683  4.660000e-01                  1   
3         θ4         2.135       1.327  1.760000e+00                  5   
4         θ5         2.492       7.235  5.234400e+01                  1   
5         θ6         0.024       0.046  2.000000e-03                  0   
6         θ7         0.010       0.030  1.000000e-03                  0   
7         θ8        10.371      30.290  9.175090e+02                  2   
8         θ9    328102.780  974942.912  9.505137e+11                  3   
9        θ10         0.0

## Base Case - Fixed Parameters

In [7]:
# --- t-value Calculation of Multiple Combinations ---
import os
param_names = [f"th{i+1}" for i in range(len(theta_nominal_completo))]

combinaciones_fijadas = [["th9"], ["th3", "th9"], ["th3", "th7", "th9"], ["th7"], ["th7", "th11"],
                         ["th7", "th11", "th12"], ["th7", "th9", "th12"], ["th4", "th7", "th12"]]  # setting the lecture parameters

calcular_t_values_multiple_combinaciones(
    residuals_func=residuals,
    theta_0=theta_nominal_completo,
    bounds=(thetaLow, thetaHigh),
    texp=t_exp,
    ydata=y_exp_interp.flatten(),
    combinaciones_fijadas=combinaciones_fijadas,
    param_names=param_names,
    sim_per_combo=10,
    n_jobs=-1
)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   8 | elapsed: 13.7min remaining: 41.2min
[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed: 13.8min remaining: 23.0min
[Parallel(n_jobs=-1)]: Done   4 out of   8 | elapsed: 14.5min remaining: 14.5min
[Parallel(n_jobs=-1)]: Done   5 out of   8 | elapsed: 14.5min remaining:  8.7min
[Parallel(n_jobs=-1)]: Done   6 out of   8 | elapsed: 14.6min remaining:  4.9min



🔒 Fijados: th9
🔓 Libres: th1, th2, th3, th4, th5, th6, th7, th8, th10, th11, th12, th13, th14
🔁 Iteraciones exitosas: 10 de 10

╒═════════════╤══════════════╤═════════╤═════════════════════╤═════════════════════════╕
│ Parametro   │   t promedio │   t std │ Significativo (#)   │ Significativo (|t|>2)   │
╞═════════════╪══════════════╪═════════╪═════════════════════╪═════════════════════════╡
│ th1         │        0.779 │   2.258 │ 1 / 10              │ —                       │
├─────────────┼──────────────┼─────────┼─────────────────────┼─────────────────────────┤
│ th2         │        0.029 │   0.047 │ 0 / 10              │ —                       │
├─────────────┼──────────────┼─────────┼─────────────────────┼─────────────────────────┤
│ th3         │        0.011 │   0.009 │ 0 / 10              │ —                       │
├─────────────┼──────────────┼─────────┼─────────────────────┼─────────────────────────┤
│ th4         │        0.044 │   0.124 │ 0 / 10              │ —      

[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed: 15.5min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed: 15.5min finished
